# All Deduplication Helper Functions

In [7]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import geopandas as gpd
import re
import unicodedata
from collections import Counter
import geohash
from shapely.geometry import Point

### NYC Blacklist + Other Mappings

In [8]:
GLOBAL_FOOD = {"pizza", "pizzeria", "cafe", "caffe", "coffee", "grill", "restaurant", "bar", "deli", "bakery", "express",
                  "market", "shop", "mart", "store", "grocery", "supermarket", "food", "gourmet", "cart", "fresh", "kitchen", 
                  "diner", "pub", "bistro", "tavern", "farm", "chicken", "burger", "sandwich", "taco", "tacos", "sushi", "noodle", "noodles"
                  "salad", "sub", "ice cream", "dessert", "breakfast", "brunch", "lunch", "dinner", "takeout", "delivery", 
                  "snack", "chocolate", "tea", "juice", "smoothie", "wine", "beer", "cocktail", "brewery", "distillery", 
                  "winery", "patisserie", "pastry", "bagel", "donut", "pancake", "waffle", "crepe", "treat", "cuisine", "truck",
                  "italian", "mexican", "chinese", "japanese", "korean", "indian", "thai", "vietnamese", "greek", "spanish",
                  "french", "american", "cuban", "cajun", "creole", "soul food", "bbq", "steakhouse", "seafood",
                  "vegetarian", "vegan", "gluten-free", "organic", "local", "artisan", "handmade", "craft", "homemade",
                  "family-owned", "authentic", "traditional", "fusion", "gastro", "gastropub", "halal", "kosher"}
GLOBAL_TRANS_ADDR = {"east", "west", "north", "south", "st", "street", "ave", "avenue", 
                 "blvd", "road", "rd", "rd.", "drive", "cab", "car", "truck", "van", "taxi", "metro", "sub", "subway", "mta", "station", "apt", "apartment", "station", "corner", "bus", "express", "line", "line", "plaza", "plz", "square", "sq", "lane", "ln", "way", "wy", "court", "ct",
                 "park", "pl", "pkwy", "parkway", "circle", "cir", "highway", "hwy", "route", "rte", "exit", "exit", "bridge", "bridges", "crossing", "crossings",
                 "crossings", "intersection", "intersections", "boulevard", "boulevards", "roadway", "roadways", "driveway", "driveways","avenue", "avenues", "streetway"}
GLOBAL_LOC = {"city", "village", "town", "museum", "group", "house", "center", "ctr", "art", "shop", "show", "theatre", "theater", "office", "service", "services", "bank", "jewelry", "club", 
              "community", "garden", "park", "field", "beach", "ocean", "river", "playground", "school", "college", "university", "library", "gallery", "studio", "hall", "auditorium", "venue", "church"
              "cleaner", "cleaners", "laundry", "laundromt", "pharmacy", "church", "clinic", "gym", "hospital", "fitness", "nail", "nails", "salon", "spa", "barber", "project", "projects"}
GLOBAL_WORDS = {"the", "a", "an", "and", "&", "of", "in", "for", "to", "at", "@", "on", "out", "with", "by", "from", "as", "that", "this", "it", "is", "was", "be", "are", "day", "care", "co."}
GLOBAL_SCHOOL_WRDS = {"high", "middle", "elementary", "school", "academy", "charter", "magnet", "daycare", "day", "care"}
GLOBAL_FIRE_STATION = {"fdny", "engine", "ems", "rescue", "group"}
GLOBAL_POLICE_STATION = {"nypd"}
LOCAL_PHRASES_NYC = {"new", "york", "nyc", "manhattan", "brooklyn", "queens", "bronx", "staten", "island", "ny", "city", "upper", "lower", "side", "marks"}
COMMON_PHRASES = GLOBAL_FOOD.union(GLOBAL_TRANS_ADDR).union(GLOBAL_LOC).union(GLOBAL_WORDS).union(LOCAL_PHRASES_NYC).union(GLOBAL_SCHOOL_WRDS).union(GLOBAL_FIRE_STATION).union(GLOBAL_POLICE_STATION)

# # Neighborhood and local area terms to ignore during name normalization
LOCAL_AREAS_NYC = [

    # Lower Manhattan / East Side
    "alphabet", "lower", "west", "north", "east", "south", "street", "side", "les",
    "two", "bridges", "chinatown", "nolita", "soho", "noho", "little", "italy",
    "bowery", "seaport", "civic", "center", "marks", "wall", "financial", "tribeca", "fidi", "delancey",
    "clinton", "canal",
    
    # Midtown / Gramercy / Chelsea
    "gramercy", "flatiron", "murray", "midtown",
    "koreatown", "garment", "district", "nomad", "chelsea", "hell", "hells",
    "hudson", "yards", "theater", "times", "square", "rockefeller", "kips", "turtle", "bay", "herald", "penn", "station", "empire",
    
    # Upper Manhattan
    "upper", "harlem", "spanish", "heights",
    "morningside", "hamilton", "inwood", "washington","manhattenville", "sugar", "hill", "dyckman", "fort", "george", "columbia",
    
    # Downtown / River Areas / Parks
    "tompkins", "river", "park",
    "stuytown", "stuyvesant", "oval", "union",
    "madison", "bryant", "central", "battery", "riverside", "fdr", "drive",
    
    # Outer Boroughs or Bordering Areas
    "brooklyn", "queens", "bushwick", "greenpoint", "williamsburg", "bed-stuy",
    "dumbo", "long island city", "astoria", "ridgewood",
]

NYC_BLACKLIST = COMMON_PHRASES.union(LOCAL_AREAS_NYC)

# Simple number word mappings up to 100 (expand if needed)
NUM_WORDS = {
    "zero": 0, "one": 1, "first": 1,
    "two": 2, "second": 2,
    "three": 3, "third": 3,
    "four": 4, "fourth": 4,
    "five": 5, "fifth": 5,
    "six": 6, "sixth": 6,
    "seven": 7, "seventh": 7,
    "eight": 8, "eighth": 8,
    "nine": 9, "ninth": 9,
    "ten": 10, "tenth": 10,
    "eleven": 11, "eleventh": 11,
    "twelve": 12, "twelfth": 12,
    "thirteen": 13, "thirteenth": 13,
    "fourteen": 14, "fourteenth": 14,
    "fifteen": 15, "fifteenth": 15,
    "sixteen": 16, "sixteenth": 16,
    "seventeen": 17, "seventeenth": 17,
    "eighteen": 18, "eighteenth": 18,
    "nineteen": 19, "nineteenth": 19,
    "twenty": 20, "twentieth": 20,
    "thirty": 30, "thirtieth": 30,
    "forty": 40, "fortieth": 40,
    "fifty": 50, "fiftieth": 50,
    "sixty": 60, "sixtieth": 60,
    "seventy": 70, "seventieth": 70,
    "eighty": 80, "eightieth": 80,
    "ninety": 90, "ninetieth": 90
}

# Normalize street suffixes
SUFFIX_MAP = {
    "st": "street", "st.": "street", "street": "street",
    "rd": "road", "rd.": "road", "road": "road",
    "ave": "avenue", "ave.": "avenue", "avenue": "avenue",
    "blvd": "boulevard", "blvd.": "boulevard", "boulevard": "boulevard",
    "ln": "lane", "ln.": "lane", "lane": "lane",
    "dr": "drive", "dr.": "drive", "drive": "drive",
    "ct": "court", "ct.": "court", "court": "court",
    "pl": "place", "pl.": "place", "place": "place"
}
def normalize_suffix(word):
    return SUFFIX_MAP.get(word.lower(), word.lower())

### FSQ Deduplication Helpers

Geohash Utils

In [12]:
def assign_geohashes(df, precision=7):
    """
    Creates a new column, 'geohash', in [df] with the geohash code for each POI row.
    
    The length of the geohash code is [precision]. Longer geohashes are more precise and thus are smaller regions.
    Precision 1 is a ~(5000km x 5000km) area (ex: large country) while Precision 7 is a ~(153m x 153m) area (ex: size of a Manhattan zip).

    df (FSQ DataFrame/GeoDataFrame): must have 'latitude' and 'longitude' columns
    precision: length of a geohash -> the size of each grid in the partition. 1 <= precision <= 12.
    """
    assert 'latitude' in df.columns and 'longitude' in df.columns
    assert precision >= 1 and precision <= 12

    df['geohash'] = df.apply(lambda row: geohash.encode(row['latitude'], row['longitude'], precision=precision), axis=1)
    return df

def get_neighboring_geohashes(hash):
    """ 
    Returns a list of [hash] and the geohash codes of [hash]'s eight immediate neighbors.
    Can be used to query surrounding areas for a POI or nearby POIs.

    Parameters:
    hash (string): geohash between length 1-12
    """
    assert len(hash) >= 1 and len(hash) <= 12
    neighbors = geohash.neighbors(hash)
    # include the original geohash
    return [hash] + neighbors

Name Utils

In [66]:
def words_to_number(tokens):
    """Convert tokens from their word form to digit-string form. 
    Ex: 
    words_to_number(['twenty', 'second']) = ['22']
    words_to_number(['seven'] = ['7'])
    """
    result = []
    i = 0
    while i < len(tokens):
        t = tokens[i]
        if t in NUM_WORDS:
            val = NUM_WORDS[t]
            j = i + 1
            if j < len(tokens) and tokens[j] in NUM_WORDS:
                comb = val + NUM_WORDS[tokens[j]]
                result.append(str(comb))
                i += 2
                continue
            result.append(str(val))
        else:
            result.append(t)
        i += 1
    return result

def capitalize_str(name):
    """ 
    Capitalizes the first letter of each word in [name].

    Parameters:
        name (str): POI name
    """
    parts = name.split()
    upper = [p.capitalize() for p in parts]
    return " ".join(upper)

def clean_name(name, lower = False):
    """ 
    Cleans a name by removing excess spaces and accents, and replacing punctuation/symbols with a space.

    Parameters:
    name (str): a POI location name
    lower (boolean): whether to make the name all lowercase or not.
    """
    if not isinstance(name, str):
        return ""
    # Collapse multiple spaces
    name = re.sub(r'\s+', ' ', name).strip()
    # Remove accents/diacritics (e.g. é → e)
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode()

    # Replace punctuation/symbols with space
    name = re.sub(r'[^a-zA-Z0-9\s]', ' ', name)
    return name.lower() if lower else name

def remove_common_words(name, blacklist = NYC_BLACKLIST):
    """
    Removes ordinal suffixes (letters attached to digits) from [name],
    converts all chars into lowercase and word numbers into their digit forms. 
    Then, filters out any tokens in [blacklist], all digits, and 1-letter words.

    Parameters:
        name (str): FSQ point of interest name
        blacklist (str list, optional): list of words to filter out of name 
        (common filler words, apt/building numbers, directions like n/w/s/e)
    """
    name = re.sub(r'\b(\d+)(st|nd|rd|th)\b', r'\1', name)
    parts = name.lower().split()
    parts = words_to_number(parts)
    parts = [p for p in parts if p not in blacklist]
    return " ".join(p for p in parts if not p.isdigit() and len(p) > 1)

def longest_common_substring(strs, min_len = 3, blacklist = NYC_BLACKLIST):
    """
    Find the longest common substring between a list of strings.
    Parameters:
        strs (list): List of 2+ strings to compare.
        min_len (int, optional): minimum length allowed for of a longest common substring
        blacklist (list, optional): words to remove from [strs] to form the common substring
    """
    assert strs is not None and len(strs) >= 2, "At least two strings are required to find a common substring."
    strs = [remove_common_words(clean_name(s, True)) for s in strs]
    shortest_str = min(strs, key = len)
    longest_substring = ""

    n = len(shortest_str)
    for i in range(n):
        for j in range(i + 1, n + 1):
            substr = shortest_str[i:j]
            if all(substr in s for s in strs) and len(substr.strip()) >= min_len:
                if len(substr) > len(longest_substring):
                    longest_substring = substr.strip()
    return longest_substring


def choose_common_name_from_group(names, blacklist = NYC_BLACKLIST):
    """ 
    Returns the representative name amongst a group of [names].
    If there exists at least one name that equals the longest common substring of the group,
    then returns the substring. 
    Otherwise, selects the first occurrence of the most frequent name in the group (using Counter).

    If the longest substring is less than 3 chars or is fully in [blacklist] it returns None.
    If all names are addresses with the same street but different numbers, it returns None.

    Parameters:
        names (str list): group of names, must not be None
        blacklist (str list): words to be filtered out and not used to determine the representative name 
    """
    assert names
    cleaned_names = [clean_name(n) for n in names]
    longest_substr = longest_common_substring(names)
    if len(longest_substr) < 3 or longest_substr.lower() in blacklist:
        return None
    
    # Compute address parts
    addr_parts = []
    for n in cleaned_names:
        parts = n.split()
        if len(parts) > 1 and parts[0].isdigit():
            num = parts[0]
            # street_tokens = [normalize_suffix(p) for p in parts[1:]]
            # street = " ".join(street_tokens)
            street = parts[1].lower()
            if street and num:
                addr_parts.append((num, street))
        # elif len(parts) > 1 and parts[-1].isdigit():
        #     num = parts[-1]
        #     prefix = parts[-2]
        #     if prefix and num:
        #         addr_parts.append((num, prefix))

    street_set = {s for _, s in addr_parts}
    number_set = {n for n,_ in addr_parts}

    # print("street_set: " + str(street_set))
    # print("num_set: " + str(number_set))
    if len(street_set) == 1 and len(number_set) > 1:
        return None

    for og_name, cleaned_name in zip(names, cleaned_names):
        bklst_filtered_name = remove_common_words(cleaned_name, blacklist)
        if bklst_filtered_name == longest_substr.lower():
            return og_name
            # return capitalize_str(og_name)
    return Counter(names).most_common(1)[0][0]
    # return capitalize_str(Counter(names).most_common(1)[0][0])


Other Utils

In [ ]:
def convert_fsq_csv_to_gdf(fsq_file, geometry_col ='geometry'):
    """ 
    Loads a FSQ Places dataset (CSV or Parquet) and converts it into a GeoDataFrame.

    Parameters:
        fsq_file (str): path to FSQ Dataset file (CSV or Parquet)
        geometry_col (str): column name used to store POI geometries
    """
    assert isinstance(fsq_file, str)
    try:
        df = pd.read_csv(fsq_file)
    except:
        df = pd.read_parquet(fsq_file)
    df[geometry_col] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry=geometry_col, crs="EPSG:4326")
    gdf['date_created'] = pd.to_datetime(gdf['date_created'], errors='coerce')
    gdf['date_closed'] = pd.to_datetime(gdf['date_closed'], errors='coerce')
    return gdf


def extract_top_category(cat_array):
    """
    Returns the top-level category of a single 'fsq_category_label' entry within a POI row.
    If cat_array is not an np.ndarray or is an empty np.ndarray, then returns ''

    Parameters:
        cat_array (np.ndarray): array of string category labels, where each label is broken into hierarchical subcategories using '<'/
    Returns: a string representing the top-level category of a label
    """
    if isinstance(cat_array, np.ndarray) and len(cat_array) > 0:
        return cat_array[0].split(' > ')[0].strip()
    else:
        return ''

def filter_by_zcta(gdf, zcta, zcta_codes):
    """
    Filters the GeoDataFrame [gdf] to include only POIs within the specified ZCTA code.
    
    Parameters:
        gdf (gpd.GeoDataFrame): GeoDataFrame containing POIs with 'geometry' column.
        zcta (gpd.GeoDataFrame): GeoDataFrame containing ZCTA polygons.
        zcta_code (str): The ZCTA code to filter by.
    
    Returns:
        gpd.GeoDataFrame: Filtered GeoDataFrame containing only POIs within the specified ZCTA.
    """

    # Make sure both CRS's are EPSG:4326 (meters)
    gdf = gdf.to_crs(epsg=4326)
    zcta = zcta.to_crs(epsg=4326)
    zcta_filtered = zcta[zcta['ZCTA5CE20'].isin(zcta_codes)]
    return gpd.sjoin(gdf, zcta_filtered, predicate = 'within')

def select_most_recent_row(df):
    """
    Take the most recent row from a group of sPOIs based on 'date_refreshed' or 'date_created'.
    If 'date_refreshed' is NaN for all rows, use 'date_created' instead.
    Parameters:
        df (pd.DataFrame): DataFrame containing a group of POIs with 'date_refreshed' and 'date_created' columns.
    """
    if df['date_refreshed'].notna().any():
        return df.sort_values(by = 'date_refreshed', ascending=False).iloc[0]
    else:
        return df.sort_values(by = 'date_created', ascending=False).iloc[0]
    
def normalize_group(df, cols):
    """ 
    Standardizes each column in [cols] in [df] and stores it in a new column, '[col]_norm', in [df].

    Parameters:
        df (DataFrame): must have at least one column
        cols (str list): list of column names in [df]
    """
    for col in cols:
        assert col in df.columns
        scaler = MinMaxScaler()
        df[col + "_norm"] = scaler.fit_transform(df[col])
    return df

def save_results_to_gdf(df, resolved_map):
    """
    Adds two new columns to [df]: 'isdup' and 'resolved_fsq_id'.
    For each row in [df], 'isdup' = True when this row is a duplicate and isdup = 0 otherwise
    For each duplicate row r in [df], 'resolved_fsq_id' maps r to its corr. row in the filtered df, that it was merged into.
    Parameters:
    df:
    dup_lst:
    """
    df['isdup'] = df['fsq_place_id'].isin(resolved_map.keys()).astype(bool)
    df['resolved_fsq_id'] = df['fsq_place_id'].map(resolved_map)
    # update for kept rows in filtered df
    kept_ids = set(resolved_map.values())
    kept = df['fsq_place_id'].isin(kept_ids)
    df.loc[kept, 'isdup'] = True
    df.loc[kept, 'resolved_fsq_id'] = df.loc[kept, 'fsq_place_id']
    return df

### Testing

In [ ]:
# not duplicates
print(choose_common_name_from_group(["21 Cameron Ave", "38 cameron ave", "45 Cameron Ave"]))
print(choose_common_name_from_group(['77 Summer Street', '87 Summer Street']))
print(choose_common_name_from_group(["20 Hemenway", "38 Hemenway Apts", "39 hemenway", "45 Hemenway Street",
                                      "Fenway Dorm 12 Hemenway St."]))
print(choose_common_name_from_group(['500 Boylston St', '501 Boylston St']))
print(choose_common_name_from_group(['12 Wendell', '14 Wendell']))
print(choose_common_name_from_group(['211 Newbury', '215 Newbury Street', 'Prince of Newbury Street']))
print(choose_common_name_from_group(['1778 Commonwealth', '1800 Commonwealth Ave']))
print(choose_common_name_from_group(['156 Pleasant St', '157 Pleasant']))
print(choose_common_name_from_group(['156 Pleasant Street', '157 Pleasant St.']))
print(choose_common_name_from_group(['41 Revere St', '70 Revere St Roof Top']))

# duplicates
print(choose_common_name_from_group(['135 Shillman Hall', 'Shillman Hall']))
print(choose_common_name_from_group(['64 Revere St', 'revere st']))
print(choose_common_name_from_group(['Hodan Management', 'Hodan Property Management & Dev']))
print(choose_common_name_from_group(['Flower Market At Star Market Quincy', 'Star Market', 'Stars at Quincy']))
print(choose_common_name_from_group(["Aldrich 007", "Aldrich 008", "Aldrich 010", "Aldrich 011", "Aldrich 107", 
"Aldrich 109", "Aldrich 110", "Aldrich 111", "Aldrich 209", "Aldrich 211", "Aldrich 9", "Aldrich Hall"
]))
print(choose_common_name_from_group(['1330 Boylston', '1330 Boylston - Luxury Apartments']))
print(choose_common_name_from_group(['WordCamp Boston 2012', 'Wordcamp Boston 2011']))
print(choose_common_name_from_group(['Lecture Hall 505', 'Lecture Hall 511']))


None
None
None
None
None
None
None
None
None
None
135 Shillman Hall
64 Revere St
Hodan Management
Star Market
Aldrich 007
1330 Boylston
WordCamp Boston 2012
Lecture Hall 505
